In [1]:
import torch 
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Create Torch DATASET


In [3]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir=img_dir
        with open(data_file, "r") as f:
            self.data= json.load(f)

        self.transforms=transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=[0.229, 0.224, 0.225])


        ])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data(index)
        img= cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w=img.shape[:2]


        #keypoints extraction
        img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img=self.transforms(img)
        kps= np.array(item['kps']).flatten()
        kps=kps.astype(np.float32)

        kps[::2] *= 224.0/w       #Adjust x 
        kps[1::2] *=224.0/h       #Adjust y

        return img, kps
    



        

In [ ]:
train_dataset = KeypointsDataset("data/images", "data/data_train.json")
val_dataset= KeypointsDataset("data/images", "data/data_val.json")


train_loader= DataLoader(train_dataset, batch_size=8, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)


In [6]:
#create model

model = models.resnet50(pretrained=True)  
model.fc = torch.nn.Linear(model.fc.in_features, 14*2)  #14 is keypoints and replace the last layer


/home/chirag/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/chirag/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
model = model.to(device)

In [8]:
#train the model 

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
epochs = 20
for epoch in range(epochs):
    for i, (imgs, kps) in enumerate(train_loader):
        imgs= imgs.to(device)
        kps= kps.to(device)

        optimizer.zero_grad()
        outputs= model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i%10 ==0:
            print(f"Epoch {epochs}, iter {i}, loss:{loss.item()}")


In [ ]:
torch.save(model.stat_dict(), "keypoints_model.pth")